In [ ]:
!pip3 uninstall -y torch torchvision torchaudio torchtext
!pip3 install --pre torch torchvision torchaudio torchtext --extra-index-url https://download.pytorch.org/whl/nightly/cpu

from google.colab import drive
drive.mount('/content/drive')

Found existing installation: torch 1.11.0+cu113
Uninstalling torch-1.11.0+cu113:
  Successfully uninstalled torch-1.11.0+cu113
Found existing installation: torchvision 0.12.0+cu113
Uninstalling torchvision-0.12.0+cu113:
  Successfully uninstalled torchvision-0.12.0+cu113
Found existing installation: torchaudio 0.11.0+cu113
Uninstalling torchaudio-0.11.0+cu113:
  Successfully uninstalled torchaudio-0.11.0+cu113
Found existing installation: torchtext 0.12.0
Uninstalling torchtext-0.12.0:
  Successfully uninstalled torchtext-0.12.0
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu
     |████████████████████████████████| 188.1 MB 74 kB/s 
     |████████████████████████████████| 13.6 MB 26.7 MB/s 
     |████████████████████████████████| 3.4 MB 836 kB/s 
     |████████████████████████████████| 1.8 MB 53.3 MB/s 
Mounted at /content/drive


## Preparation

First we import the necessary packages, and fetch data that we work on.




In [ ]:
# %matplotlib inline

import os
import time
from typing import List

import IPython
import matplotlib
import matplotlib.pyplot as plt
import requests
import torch
import torchaudio
import torch.nn.functional as F
import glob

print(os.getcwd())

matplotlib.rcParams["figure.figsize"] = [16.0, 4.8]

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(torchaudio.__version__)
print(device)

speech_dir = '/content/drive/MyDrive/wav/F02' # 这里需要修改成自己的音频存放路径 wav path
in_file = '/content/drive/MyDrive/transcript/speaker_wordlist.xlsx' # 这里需要改成自己的标签文件 labelled path
out_file = '/content/drive/MyDrive/wav/F02.xls' # 输出的识别结果 result path

files = []
if not os.path.exists(speech_dir):
  raise FileNotFoundError('not exsist')
else:
  for file in glob.glob(os.path.join(speech_dir, '**', '*.wav'), recursive=True):
    files.append(file)

/content
1.12.0.dev20220519+cpu
0.12.0.dev20220519+cpu
cpu


# Decoder

In [ ]:
from torchaudio.prototype.ctc_decoder import download_pretrained_files
from torchaudio.prototype.ctc_decoder import lexicon_decoder

# CTC Greedy Decode
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission: torch.Tensor) -> List[str]:
        """Given a sequence emission over labels, get the best path
        Args:
          emission (Tensor): Logit tensors. Shape `[num_seq, num_label]`.

        Returns:
          List[str]: The resulting transcript
        """
        indices = torch.argmax(emission, dim=-1)  # [num_seq,]
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        joined = "".join([self.labels[i] for i in indices])
        return joined.replace("|", " ").strip().split()

## LM and Lexicon






In [ ]:
from torchaudio.prototype.ctc_decoder import download_pretrained_files
from torchaudio.prototype.ctc_decoder import lexicon_decoder
import xlrd
import xlwt

print('Downloading N-Gram Language Model')
ngram_files = download_pretrained_files("librispeech-4-gram")
print(ngram_files)

print('Download Acoustic Model')
# bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
bundle = torchaudio.pipelines.WAV2VEC2_ASR_LARGE_LV60K_960H
model = bundle.get_model().to(device)

labels = bundle.get_labels()
tokens = [label.lower() for label in bundle.get_labels()]
print('Tokens:{}'.format(tokens))

# Greedy Decoder
greedy_decoder = GreedyCTCDecoder(tokens)

# Beamsearch Decoder
LM_WEIGHT = 3.23
WORD_SCORE = -0.26
beam_search_decoder = lexicon_decoder(
    lexicon=ngram_files.lexicon,
    tokens=ngram_files.tokens,
    lm=ngram_files.lm,
    nbest=3,
    beam_size=1500,
    lm_weight=LM_WEIGHT,
    word_score=WORD_SCORE,
)


# 读取excel，获取音频标签
label_dict = {}

book = xlrd.open_workbook(in_file)
sheet = book.sheets()[0]
nrows = sheet.nrows
print('Num rows : ', nrows)
ncols = sheet.ncols
print('Num Cols : ', ncols)

print('First row is {}'.format(sheet.row_values(0)))
for i in range(1, nrows):
  arr = sheet.row_values(i)
  if len(arr) != 2:
    print('The col numbers not equal to 2!')
    continue
  label, key = arr
  if key not in label_dict:
    label_dict[key] = label
  else:
    print('label_dict already has label_dict[{}]={}'.format(key, label_dict[key]))



  0%|          | 0.00/4.97M [00:00<?, ?B/s]

  0%|          | 0.00/57.0 [00:00<?, ?B/s]

  0%|          | 0.00/2.91G [00:00<?, ?B/s]

PretrainedFiles(lexicon='/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lexicon.txt', tokens='/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/tokens.txt', lm='/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lm.bin')
Download Acoustic Model


Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_large_lv60k_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_large_lv60k_asr_ls960.pth


  0%|          | 0.00/1.18G [00:00<?, ?B/s]

Tokens:['-', '|', 'e', 't', 'a', 'o', 'n', 'i', 'h', 's', 'r', 'd', 'l', 'u', 'm', 'w', 'c', 'f', 'g', 'y', 'p', 'b', 'v', 'k', "'", 'x', 'j', 'q', 'z']


/usr/local/lib/python3.7/dist-packages/torchaudio/prototype/ctc_decoder/_ctc_decoder.py:313: UserWarning: `lexicon_decoder` is now deprecated. Please use `ctc_decoder` instead.
  warnings.warn("`lexicon_decoder` is now deprecated. Please use `ctc_decoder` instead.")


Num rows :  456
Num Cols :  2
First row is ['WORD', 'FILE NAME']


#Writing in the excel

In [ ]:
total_word = 0
total_char = 0
error_word = 0
error_char = 0

# 写excel
wtbook = xlwt.Workbook(encoding = 'utf-8')
wtsheet = wtbook.add_sheet('Sheet1', cell_overwrite_ok=True)
wtsheet.write(0, 0, 'WORD')
wtsheet.write(0, 1, 'PRED')
wtsheet.write(0, 2, 'FILE NAME')

with torch.inference_mode(), open(out_file, 'w') as fw:
  for i,file in enumerate(files):
    print('Processing {}'.format(file))
    dirname,filename = os.path.split(file)
    basename,ext = os.path.splitext(filename) 
    arr = basename.split('_')
    if arr[2].find('UW') != -1: # 带UW
      key = '_'.join(arr[1:3])
    else:
      key = arr[2]
    
    if key not in label_dict:
      print('{} not in label_dict'.format(key))
      continue

    label_word_list = [label_dict[key].lower().replace(' ', '')] # as one word
    label_char_list = list(label_dict[key].lower().replace(' ', ''))

    waveform, sample_rate = torchaudio.load(file)
    if sample_rate != bundle.sample_rate:
      waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

    emissions, _ = model(waveform.to(device))
    emission = emissions.cpu().detach()

    # greedy_result = greedy_decoder(emission[0]) # list
    # print('Greedy decode result:{}'.format(greedy_result))

    # greedy_word_list = greedy_result
    # greedy_char_list = list(''.join(greedy_result).replace(' ', ''))
    # error_word += torchaudio.functional.edit_distance(label_word_list, greedy_word_list)
    # total_word += len(label_word_list)
    # error_char += torchaudio.functional.edit_distance(label_char_list, greedy_char_list)
    # total_char += len(label_char_list)

    beam_search_result = beam_search_decoder(emission)[0][0].words
    print('Beam search result:{}'.format(beam_search_result))

    beam_search_word_list = [''.join(beam_search_result).replace(' ', '')]
    beam_search_char_list = list(''.join(beam_search_result).replace(' ', ''))
    error_word += torchaudio.functional.edit_distance(label_word_list, beam_search_word_list)
    total_word += len(label_word_list)
    error_char += torchaudio.functional.edit_distance(label_char_list, beam_search_char_list)
    total_char += len(label_char_list)

    wtsheet.write(i+1, 0, label_dict[key].lower().replace(' ', ''))
    wtsheet.write(i+1, 1, ''.join(beam_search_result).replace(' ', ''))
    wtsheet.write(i+1, 2, basename)

wtbook.save(out_file)
print('Error Char:{},Total Char:{}'.format(error_char, total_char))
print('Total CER:{:.2f}'.format(error_char/total_char))
print('Error Word:{},Total Word:{}'.format(error_word, total_word))
print('Total WER:{:.3f}'.format(error_word/total_word))

Processing /content/drive/MyDrive/wav/F02/F02_B1_LB_M4.wav
Beam search result:['one']
Processing /content/drive/MyDrive/wav/F02/F02_B1_CW56_M5.wav
Beam search result:['ten']
Processing /content/drive/MyDrive/wav/F02/F02_B1_C10_M3.wav
Beam search result:['on']
Processing /content/drive/MyDrive/wav/F02/F02_B1_CW77_M4.wav
Beam search result:['cold']
Processing /content/drive/MyDrive/wav/F02/F02_B1_D7_M8.wav
Beam search result:['something']
Processing /content/drive/MyDrive/wav/F02/F02_B1_LH_M3.wav
Beam search result:['town']
Processing /content/drive/MyDrive/wav/F02/F02_B1_C3_M4.wav
Beam search result:[]
Processing /content/drive/MyDrive/wav/F02/F02_B1_D4_M5.wav
Beam search result:['fall']
Processing /content/drive/MyDrive/wav/F02/F02_B1_C4_M2.wav
Beam search result:['in', 'toil']
Processing /content/drive/MyDrive/wav/F02/F02_B1_CW56_M2.wav
Beam search result:['ten']
Processing /content/drive/MyDrive/wav/F02/F02_B1_C16_M8.wav
Beam search result:['a', 'd']
Processing /content/drive/MyDrive